In [1]:
%load_ext jupyter_black

In [2]:
import os

import torch
import numpy as np

import mesoscaler as ms


_local_data = os.path.abspath("data")

urma_store = os.path.join(_local_data, "urma.zarr")
assert os.path.exists(urma_store)
era5_store = os.path.join(_local_data, "era5.zarr")
assert os.path.exists(era5_store)

from mesoscaler.enums import (
    # - ERA5
    GEOPOTENTIAL,
    SPECIFIC_HUMIDITY,
    TEMPERATURE,
    U_COMPONENT_OF_WIND,
    V_COMPONENT_OF_WIND,
    # - URMA
    SURFACE_PRESSURE,
    TEMPERATURE_2M,
    SPECIFIC_HUMIDITY_2M,
    U_WIND_COMPONENT_10M,
    V_WIND_COMPONENT_10M,
    SURFACE_PRESSURE,
)

era5_dvars = [
    GEOPOTENTIAL,
    TEMPERATURE,
    SPECIFIC_HUMIDITY,
    U_COMPONENT_OF_WIND,
    V_COMPONENT_OF_WIND,
]

urma_dvars = [
    SURFACE_PRESSURE,
    TEMPERATURE_2M,
    SPECIFIC_HUMIDITY_2M,
    U_WIND_COMPONENT_10M,
    V_WIND_COMPONENT_10M,
]

dataset_sequence = ms.open_datasets([(urma_store, urma_dvars), (era5_store, era5_dvars)])
dataset_sequence

DatasetSequence(size=2)[
<xarray.DependentDataset>
Dimensions:               (Y: 1597, X: 2345, T: 24, Z: 1)
Coordinates:
    latitude              (Y, X) float64 19.23 19.23 19.24 ... 54.38 54.38 54.37
    longitude             (Y, X) float64 233.7 233.7 233.8 ... 300.9 300.9 301.0
  * time                  (T) datetime64[ns] 2019-01-01T03:00:00 ... 2019-01-...
  * vertical              (Z) float64 1.013e+03
Dimensions without coordinates: Y, X, T, Z
Data variables:
    specific_humidity_2m  (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    surface_pressure      (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    temperature_2m        (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    u_wind_component_10m  (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    v_wind_component_10m  (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
Attributes:
    depends:  Dependencies(URMA)
<xarray.DependentDataset>
Dimensions:              (T: 25, Z: 23, Y: 721, X: 1440)
Coordinates:
    latitude             (Y, X) float32 90.0 90.0 90.0 ... -90.0 -90.0 -90.0
  * vertical             (Z) int64 200 225 250 300 350 ... 900 925 950 975 1000
    longitude            (Y, X) float32 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time                 (T) datetime64[ns] 2019-01-01 ... 2019-01-02
Dimensions without coordinates: T, Z, Y, X
Data variables:
    geopotential         (T, Z, Y, X) float32 dask.array<chunksize=(1, 23, 721, 1440), meta=np.ndarray>
    specific_humidity    (T, Z, Y, X) float32 dask.array<chunksize=(1, 23, 721, 1440), meta=np.ndarray>
    temperature          (T, Z, Y, X) float32 dask.array<chunksize=(1, 23, 721, 1440), meta=np.ndarray>
    u_component_of_wind  (T, Z, Y, X) float32 dask.array<chunksize=(1, 23, 721, 1440), meta=np.ndarray>
    v_component_of_wind  (T, Z, Y, X) float32 dask.array<chunksize=(1, 23, 721, 1440), meta=np.ndarray>
Attributes:
    depends:  Dependencies(ERA5)
]

In [3]:
from model.mae import MaskedAutoencoder3d

width = 80
height = 40
distance_ratio = 2.5  # km
patch_ratio = 0.2

patch_size = (
    int(width * patch_ratio),
    int(height * patch_ratio),
)

dx = int(width * distance_ratio)
dy = int(height * distance_ratio)

levels = [
    1013.25,
    1000,
    925,
    850,
]
img_size = (width, height)
texas = aoi = -106.6, 25.8, -93.5, 36.5

config = {
    "img_size": img_size,
    "patch_size": patch_size,
}
config, dx, dy

({'img_size': (80, 40), 'patch_size': (16, 8)}, 200, 100)

In [4]:
scale = ms.Mesoscale(dx, dy, levels=levels)
scale

Mesoscale(size=4):
-       scale: [1.00 3.81 7.52 9.43]
- levels[hPa]: [1013.25 1000.00  925.00  850.00]
-  extent[km]: 
[[ -200.00  -100.00   200.00   100.00]
 [ -761.70  -380.85   761.70   380.85]
 [-1504.23  -752.11  1504.23   752.11]
 [-1885.10  -942.55  1885.10   942.55]]

In [12]:
from torch.utils.data import DataLoader, IterableDataset


class Dataset(IterableDataset):
    def __init__(self, scale: ms.Mesoscale) -> None:
        super().__init__()

        self.resampler = resampler = scale.resample(dataset_sequence)
        self.indices = ms.AreaOfInterestSampler(
            resampler.domain, 
            aoi=(-106.6, 25.8, -93.5, 36.5),
        )

    def __iter__(self):
        for (lon, lat), time in self.indices:
            yield self.resampler(lon, lat, time)


ds = Dataset(scale)

In [14]:
b = next(iter(ds))
b.shape

(5, 1, 4, 80, 80)